# DFT and FFT

https://en.wikipedia.org/wiki/Discrete_Fourier_transform

DFT 定义：对于 $N$ 个点的输入信号 $x[n], n = 0, 1, \ldots, N-1$

$$X[k] = \sum_{n=0}^{N-1} x[n] e^{-j 2 \pi  n k / N} \quad \mathrm{for}\ k = 0, 1, \ldots, N-1$$

$$X[k] = \sum_{n=0}^{N-1} x[n] w_N^{nk}  \quad \mathrm{where} \quad w_N = e^{-j 2 \pi / N} $$

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal

import librosa as rosa
import librosa.display
from IPython.display import Audio

from ipywidgets import interact

np.set_printoptions(suppress=True)

def draw_resp_stem(x, y):
  plt.stem(x)
  (markerline, _, _) = plt.stem(y, linefmt='r', markerfmt='ro')
  markerline.set_markerfacecolor('none')

1. 电子系教材的传统讲法：Fourier 系数 ⇒ Fourier 变换 ⇒ DTFT ⇒ DFT 
https://en.wikipedia.org/wiki/Fourier_series

|     | 周期 | 非周期 | 课程 | 数学工具 |
| --- | ---- | ---- | --- | --- |
| 连续 | Fourier 系数 |  Fourier 变换 | 《信号与系统》 | 积分 |
| 离散 | DFT | DTFT | 《数字信号处理》 | 求和（向量内积）|

这种讲法对于非 EE 专业的人来说，路径太长了。现在数字信号处理应用非常广泛，很多非 EE 专业的人也需要学习 DSP （非指 DSP 芯片），特别是可能用到 FFT 来提取频率特征，例如在 Web 页面上显示音乐的视觉特效。我认为对于这部分人，可以在学过线性代数的基础上，用矩阵乘法的概念直接讲 DFT/FFT，跳过前面的 DTFT 等铺垫，用一两个小时的时间掌握 FFT 的精髓。

DFT 叫离散 Fourier 变换，其实或许叫“离散 Fourier 级数”更合适，因为它得到的是一个数组，而不是连续函数。FFT 是计算 DFT 的快速算法，但是因为估计没有人会按 DFT 的定义去算，那样太慢了，于是 DFT 和 FFT 往往可以不加区分地使用。

2. 线性代数，矩阵乘以向量 $\Rightarrow$ 酉空间的坐标变换 https://www.youtube.com/watch?v=M0Sa8fLOajA
3. 信号的相关性，$X(\omega) = \langle \boldsymbol x, \boldsymbol s_\omega \rangle$ ，$ \boldsymbol s_\omega $ 是角频率 $\omega$ 的 complex sinusoid 信号
4. 《算法导论》独辟蹊径的讲法：多项式乘法（卷积）的快速算法 https://www.youtube.com/watch?v=h7apO7q16V0
5. 把 DFT 视为单层人工神经网络 
$y_k = \varphi\left(\displaystyle \sum_n w_{kn} x_n + b_k\right)$
https://sidsite.com/posts/fourier-nets/ 
> We can consider the discrete Fourier transform (DFT) to be an artificial neural network: it is a single layer network, with **no bias**, **no activation function**, and particular values for the **weights**. The number of output nodes is equal to the number of frequencies we evaluate.

叠加两个信号很容易，如何把它分解为原始信号？

In [3]:
@interact(amp=(0, 1, 0.1), phi=(-1,1,0.1))
def synthesis(amp = 1, phi=0):
  w = np.linspace(0, 4*np.pi, 100)
  x1 = np.sin(w)
  plt.plot(w, x1)
  x2 = np.sin(2*w + phi*np.pi) * amp
  plt.plot(w, x2)
  plt.plot(w, x1+x2)
  plt.ylim(-2,2)

interactive(children=(FloatSlider(value=1.0, description='amp', max=1.0), FloatSlider(value=0.0, description='…

## Spot Correlation

对于实数，$\boldsymbol a, \boldsymbol b$ 两个向量的内积 $\langle \boldsymbol a, \boldsymbol b\rangle = \boldsymbol a \cdot \boldsymbol b = \displaystyle \sum_ i a_i b_i$

$\boldsymbol a \cdot \boldsymbol b$ 表示 $\boldsymbol a$ 在 $\boldsymbol b$ 上的投影长度。如果 $\boldsymbol a, \boldsymbol b$ 同向或反向，内积最大，如果 $\boldsymbol a, \boldsymbol b$ 垂直（正交），内积为 0.

怎么知道信号 $\boldsymbol{x} = (x_0, x_1, \ldots, x_{N-1})$ 有哪些频率成分？

一个 naive 的想法：用一个已知频率为 $\omega=2\pi f / N$ 的信号 $\boldsymbol{s}_\omega = \cos(\omega n)$ 去试，乘出来的结果越大，说明信号 $\boldsymbol{x}$ 在这个频率上的分量越大，也就是 $\boldsymbol{x}$ 和 $\boldsymbol{s}_\omega$ 的“相关性”强。

但是，就算 $\boldsymbol{x}$ 的频率也是 $\omega$，
但不巧 $\boldsymbol x = \sin(\omega n)$ ，
结果 $ \boldsymbol x \cdot \boldsymbol{s}_\omega = 0$ 咋办？


## DFT Matrix

https://en.wikipedia.org/wiki/DFT_matrix

## Peak detection and correction

Richard G. Lyons, _Understanding Digital Signal Processing 3/e_, 2011.

§ 13.15 Spectral Peak Location Algorithm


## 频率分辨率和时间分辨率

FFT 的频率分辨率 $\dfrac{f_s}{N}$，时间分辨率 $\dfrac{N}{f_s}$
